In [ ]:
import numpy as np
import keras
import tensorflow as tf
import PIL
import cv2

my_model = keras.saving.load_model('equation_solver.keras', custom_objects=None, compile=True, safe_mode=True)

my_model.summary()

In [ ]:
IMG_WIDTH = 45
IMG_HEIGHT = 45
class_names = ['!', '(', ')', '+', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'C', 'Delta', 'G', 'H', 'M', 'N', 'R', 'S', 'T', 'X', '[', ']', 'alpha', 'ascii_124', 'b', 'beta', 'cos', 'd', 'div', 'e', 'exists', 'f', 'forall', 'forward_slash', 'gamma', 'geq', 'gt', 'i', 'in', 'infty', 'int', 'j', 'k', 'l', 'lambda', 'ldots', 'leq', 'lim', 'log', 'lt', 'mu', 'neq', 'o', 'p', 'phi', 'pi', 'pm', 'prime', 'q', 'rightarrow', 'sigma', 'sin', 'sqrt', 'sum', 'tan', 'theta', 'times', 'u', 'v', 'w', 'y', 'z', '{', '}']

In [33]:
img = PIL.Image.open('examples/seven.png').convert('L')
img = img.resize((IMG_WIDTH, IMG_HEIGHT))
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = my_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
This image most likely belongs to 7 with a 96.81 percent confidence.


:)

In [34]:
image = cv2.imread('examples/2plus2.jpg', 0)
cv2.imshow('Original', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)

cv2.imshow('Binary Image', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])
contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(contour_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('Contours', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
filtered_rectangles = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    new_rect = (x, y, x + w, y + h)

    overlap = False
    for rect in filtered_rectangles:
        if (new_rect[0] < rect[2] and new_rect[2] > rect[0] and
            new_rect[1] < rect[3] and new_rect[3] > rect[1]):
            overlap = True
            break
    
    if not overlap:
        filtered_rectangles.append(new_rect)

contour_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for rect in filtered_rectangles:
    cv2.rectangle(contour_image, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)

cv2.imshow('Bounding Rectangles', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Resize all the remaining bounding rectangle to 28 by 28.
# Using our model, predict the corresponding digit/symbol for each bounding rectangle and store it in a string.